In [ ]:
import numpy as np 
import pandas as pd 
import os
from keras.callbacks import EarlyStopping

label = pd.read_csv(os.getcwd()+ "/labels.csv")
label['id1'] = label.id+'.jpg'

import tensorflow as tf
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale          =1./255,
    horizontal_flip  = True,
    validation_split = 0.20
)

train_generator224=datagen.flow_from_dataframe(
    dataframe = label,
    directory=os.getcwd()+ "/train/",
    x_col="id1",
    y_col="breed",
    color_mode = 'rgb',
    subset = 'training',
    batch_size=256,
    seed=42,
    shuffle=False,
    class_mode="categorical",
    target_size=(224,224)
)

val_generator300=datagen.flow_from_dataframe(
    dataframe = label,
    directory=os.getcwd()+ "/train/",
    x_col="id1",
    y_col="breed",
    color_mode = 'rgb',
    subset = 'validation',
    batch_size=256,
    seed=42,
    shuffle=False,
    class_mode="categorical",
    target_size=(300,300)
)
train_generator331=datagen.flow_from_dataframe(
    dataframe = label,
    directory=os.getcwd()+ "/train/",
    x_col="id1",
    y_col="breed",
    color_mode = 'rgb',
    subset = 'training',
    batch_size=256,
    seed=42,
    shuffle=False,
    class_mode="categorical",
    target_size=(331,331)
)

val_generator299=datagen.flow_from_dataframe(
    dataframe = label,
    directory=os.getcwd()+ "/train/",
    x_col="id1",
    y_col="breed",
    color_mode = 'rgb',
    subset = 'validation',
    batch_size=256,
    seed=42,
    shuffle=False,
    class_mode="categorical",
    target_size=(299,299)
)

In [18]:
import numpy as np 
import pandas as pd 
import os
from keras.callbacks import EarlyStopping

In [19]:
mnv = tf.keras.applications.inception_v3.InceptionV3(include_top = False,
                                           weights      = 'imagenet',
                                           input_shape = (300,300,3))
mnv.trainbale = False
for layer in mnv.layers:
    layer.trainable = False
    
modelInv3 = tf.keras.Sequential([
    mnv,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation ='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(256,activation ='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(120,activation = 'softmax')
])



modelInv3.compile(loss = 'categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(),
              metrics   = ['accuracy'])



In [20]:
pretrained_model = tf.keras.applications.ResNet50(include_top=False ,
                                                     weights='imagenet',  
                                                     input_shape=(300,300,3))
pretrained_model.trainable = False
    
modelRn = tf.keras.Sequential([ 
        pretrained_model,   
        tf.keras.layers.GlobalAveragePooling2D(),
        #tf.keras.layers.Dense(256, activation='relu'),
        #tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(120, activation='softmax')
        ])

modelRn.compile(loss = 'categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(),
              metrics   = ['accuracy'])



In [21]:
pretrained_modelnnl = tf.keras.applications.NASNetLarge(include_top=False ,
                                                     weights='imagenet',  
                                                     input_shape=(331,331,3))
pretrained_modelnnl.trainable = False
    
modelnnl = tf.keras.Sequential([ 
        pretrained_modelnnl,   
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(120, activation='softmax')
    ])

opt=tf.keras.optimizers.SGD(lr=1e-3, momentum=0.9)

modelnnl.compile(optimizer = opt ,
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [22]:

modelnnl.load_weights("NASNetLarge_15_epoch.h5")

#modelRn.load_weights("ResNet50.h5")
#modelInv3.load_weights("IncepV3.h5")

In [ ]:
datagenX = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale          =1./255
)
test_dir = os.getcwd()+ "/test"
generator = datagenX.flow_from_directory(
        '.',
        target_size=(331, 331),
        batch_size=256,
        color_mode = 'rgb',
        classes=['test'],  # only data, no labels
        shuffle=False
)
filenames = generator.filenames
nb_samples = len(filenames)
nb_samples

In [ ]:
predict = modelnnl.predict(generator,steps = 
                                   np.ceil(nb_samples/256))

In [ ]:
predict = modelnnl.evaluate(generator,steps = 
                                   np.ceil(nb_samples/256))
predict